In [9]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.init as init
# from torchvision.models.utils import load_state_dict_from_url
from torch.utils import model_zoo
from typing import Any
from torchvision import datasets, transforms
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torch.utils.tensorboard import SummaryWriter

import math
import random

__all__ = ['SqueezeNet', 'squeezenet1_0', 'squeezenet1_1']

model_urls = {
    'squeezenet1_0': 'https://download.pytorch.org/models/squeezenet1_0-a815701f.pth',
    'squeezenet1_1': 'https://download.pytorch.org/models/squeezenet1_1-f364aa15.pth',
}


class Fire(nn.Module):

    def __init__(
        self,
        inplanes: int,
        squeeze_planes: int,
        expand1x1_planes: int,
        expand3x3_planes: int
    ) -> None:
        super(Fire, self).__init__()
        self.inplanes = inplanes
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes,
                                   kernel_size=1)
        self.expand1x1_activation = nn.ReLU(inplace=True)
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes,
                                   kernel_size=3, padding=1)
        self.expand3x3_activation = nn.ReLU(inplace=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.squeeze_activation(self.squeeze(x))
        return torch.cat([
            self.expand1x1_activation(self.expand1x1(x)),
            self.expand3x3_activation(self.expand3x3(x))
        ], 1)


class SqueezeNet(nn.Module):

    def __init__(
        self,
        version: str = '1_0',
        num_classes: int = 1000
    ) -> None:
        super(SqueezeNet, self).__init__()
        self.num_classes = num_classes
        if version == '1_0':
            self.features = nn.Sequential(
                nn.Conv2d(3, 96, kernel_size=7, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(96, 16, 64, 64),
                Fire(128, 16, 64, 64),
                Fire(128, 32, 128, 128),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(256, 32, 128, 128),
                Fire(256, 48, 192, 192),
                Fire(384, 48, 192, 192),
                Fire(384, 64, 256, 256),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(512, 64, 256, 256),
            )
        elif version == '1_1':
            self.features = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=3, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(64, 16, 64, 64),
                Fire(128, 16, 64, 64),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(128, 32, 128, 128),
                Fire(256, 32, 128, 128),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(256, 48, 192, 192),
                Fire(384, 48, 192, 192),
                Fire(384, 64, 256, 256),
                Fire(512, 64, 256, 256),
            )
        else:
            # FIXME: Is this needed? SqueezeNet should only be called from the
            # FIXME: squeezenet1_x() functions
            # FIXME: This checking is not done for the other models
            raise ValueError("Unsupported SqueezeNet version {version}:"
                             "1_0 or 1_1 expected".format(version=version))

        # Final convolution is initialized differently from the rest
        final_conv = nn.Conv2d(512, self.num_classes, kernel_size=1)
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            final_conv,
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m is final_conv:
                    init.normal_(m.weight, mean=0.0, std=0.01)
                else:
                    init.kaiming_uniform_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.classifier(x)
        return torch.flatten(x, 1)


def _squeezenet(version: str, pretrained: bool, progress: bool, **kwargs: Any) -> SqueezeNet:
    model = SqueezeNet(version, **kwargs)
    if pretrained:
        if version == '1_0':
            model.load_state_dict(model_zoo.load_url('https://download.pytorch.org/models/squeezenet1_0-a815701f.pth'))
        if version == '1_1':
            model.load_state_dict(model_zoo.load_url('https://download.pytorch.org/models/squeezenet1_1-f364aa15.pth'))
    return model


def squeezenet1_0(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> SqueezeNet:
    r"""SqueezeNet model architecture from the `"SqueezeNet: AlexNet-level
    accuracy with 50x fewer parameters and <0.5MB model size"
    <https://arxiv.org/abs/1602.07360>`_ paper.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _squeezenet('1_0', pretrained, progress, **kwargs)


def squeezenet1_1(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> SqueezeNet:
    r"""SqueezeNet 1.1 model from the `official SqueezeNet repo
    <https://github.com/DeepScale/SqueezeNet/tree/master/SqueezeNet_v1.1>`_.
    SqueezeNet 1.1 has 2.4x less computation and slightly fewer parameters
    than SqueezeNet 1.0, without sacrificing accuracy.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _squeezenet('1_1', pretrained, progress, **kwargs)



#Accelerate the training
torch.backends.cudnn.benchmark = True
torch.manual_seed(0)
torch.cuda.manual_seed(0)


print("Target domain: {}".format('Real world'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model

model = squeezenet1_0(pretrained=True)

# Replace the final layer
model.classifier[1] = nn.Conv2d(512, 65, kernel_size=1)

# Freeze pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# Only the final layer is trainable
for param in model.classifier[1].parameters():
    param.requires_grad = True

model = model.to(device)

print(model)




## Mount Google Drive Data (If using Google Colaboratory)
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
except:
    print("Mounting Failed.")


# Data loaders
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.4),
    transforms.RandomGrayscale(0.1),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(224,224)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


directory = '~/Downloads/OfficeHomeDataset_10072016'
concatenated_datasets = []
total_size = 0

for name in ["Art","Clipart","Product"]:
    dataset = datasets.ImageFolder(
        f'{directory}/{name}',
        transform=train_transform
    )
    total_size += len(dataset)
    concatenated_datasets.append(dataset)

combined_dataset = ConcatDataset(concatenated_datasets)
indices = list(range(total_size))
random.shuffle(indices)
random_subset = Subset(combined_dataset, indices)
source_loader = DataLoader(random_subset, batch_size=64, shuffle=True)

target_dataset = datasets.ImageFolder(
    f'{directory}/Real World',
    transform=val_transform)
target_loader = torch.utils.data.DataLoader(target_dataset , batch_size=64, shuffle=True)




# Optimizer and scheduler

params = model.parameters()

optimizer = optim.SGD(params, weight_decay=.0005, momentum=.9, nesterov=False, lr=0.01)
#optimizer = optim.Adam(params, lr=lr)
step_size = int(20 * .8)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size)

print("Step size: %d" % step_size)

# Training and testing loop
results = {"training_loss": [], "training_acc": [], "test_loss": [], "test_acc": []}
criterion = nn.CrossEntropyLoss()

writer = SummaryWriter('test')
global_step = 0

# keep track of loss
training_losses = []
training_accs = []
test_losses = []
test_accs = []

# Early Stopping
epochs_early_stopping = 20
epochs_min_change = 0.0001
best_test_loss = float('inf')


Target domain: Real world
SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True

In [10]:

for now_epoch in range(20):

    lr = scheduler.get_lr()[0]


    # Training
    model.train()
    epoch_train_loss = 0.0
    epoch_train_accs = 0.0
    train_total = len(source_loader)
    for it, batch in enumerate(source_loader):

        data, labels = batch

        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()

        data_fliped = torch.flip(data, (3,)).detach().clone()
        data = torch.cat((data, data_fliped))
        labels_labels_flip = torch.cat((labels, labels))

        class_logit = model(data)
        # class_logit = model(data)
        class_loss = criterion(class_logit, labels_labels_flip)
        _, cls_pred = class_logit.max(dim=1)
        loss = class_loss

        loss.backward()
        optimizer.step()

        class_loss_train = class_loss.item()
        accuracy = torch.sum(cls_pred == labels_labels_flip.data).item() / data.shape[0]
        print(accuracy)
        print(class_loss_train)
        writer.add_scalar('Loss/Train', class_loss_train, global_step)
        writer.add_scalar('Accuracy/Train', accuracy, global_step)
        global_step += 1

        epoch_train_loss += class_loss_train
        epoch_train_accs += accuracy

        del loss, class_loss, class_logit

    epoch_train_loss = epoch_train_loss / train_total
    epoch_train_accs = epoch_train_accs / train_total

    if(now_epoch % 1 == 0):
        print(f"Training epoch {now_epoch}/{20} - "
              f"loss: {epoch_train_loss} - acc: {epoch_train_accs * 100 : 2f}%")

    training_losses.append(epoch_train_loss)
    training_accs.append(epoch_train_accs)

    scheduler.step()
    writer.add_scalar('Learning Rate', lr, now_epoch)
    # Testing
    model.eval()
    epoch_test_loss = 0.0
    epoch_test_accs = 0.0
    with torch.no_grad():
        class_correct = 0
        total = len(target_loader)
        for it, batch in enumerate(target_loader):
            data, labels = batch

            data, labels = data.to(device), labels.to(device)

            class_logit = model(data)
            # class_logit = model(data, labels, False)
            test_loss = criterion(class_logit, labels)
            _, cls_pred = class_logit.max(dim=1)

            accuracy = torch.sum(cls_pred == labels.data) / data.shape[0]
            print("testing" + str(accuracy))
            epoch_test_loss += test_loss
            epoch_test_accs += accuracy

        epoch_test_loss = epoch_test_loss / total
        epoch_test_accs = epoch_test_accs / total

        writer.add_scalar('Test/Accuracy', epoch_test_accs, now_epoch)
        if(now_epoch % 1 == 0):
            print(f"Testing - acc: {epoch_test_accs : 2f}%")

    test_losses.append(epoch_test_loss)
    test_accs.append(epoch_test_accs)

    if epoch_test_loss < (best_test_loss - epochs_min_change):
        print("Test Loss Improved from {:.4f} to {:.4f}".format(best_test_loss, epoch_test_loss))
        best_test_loss = epoch_test_loss
        epochs_early_stopping = 20
        torch.save(model.state_dict(), './squeeze_model_best2.pth')
        print("Model saved as ./squeeze_model_best2.pth")
    else:
        epochs_early_stopping -= 1
        print(f"No improvement in test loss for {20 - epochs_early_stopping} consecutive epochs.")

    # Check if Early Stopping is needed
    if epochs_early_stopping == 0:
        print(f"Early stopping after {now_epoch + 1} epochs!")
        break



results['training_loss'] = training_losses
results['training_acc'] = training_accs
results['test_loss'] = test_losses
results['test_accuracy'] = test_accs





torch.save(model.state_dict(), './squeeze_model_best2.pth')






0.015625
4.912208080291748
0.0234375
4.708477973937988
0.046875
4.485867500305176
0.0546875
4.750092029571533
0.0390625
4.647277355194092
0.0390625
4.342755317687988
0.0625
4.193538188934326
0.046875
4.421520233154297
0.171875
3.720942258834839
0.0703125
4.010695934295654
0.109375
3.945255756378174
0.1015625
3.795032262802124
0.1796875
3.494990348815918
0.1796875
3.726388692855835
0.1796875
3.370434522628784
0.21875
3.331641912460327
0.1796875
3.400785446166992
0.171875
3.7959392070770264
0.140625
3.6688454151153564
0.1796875
3.578254222869873
0.1953125
3.456599473953247
0.234375
3.38793683052063
0.3671875
2.873603105545044
0.28125
3.1445441246032715
0.3046875
2.9700355529785156
0.203125
3.4428114891052246
0.390625
2.353821039199829
0.1875
3.25038480758667
0.265625
3.278797149658203
0.328125
2.992982864379883
0.3125
2.7137041091918945
0.3046875
2.8339548110961914
0.21875
3.2073171138763428
0.375
2.8464279174804688
0.3515625
2.910618543624878
0.3046875
3.1251299381256104
0.296875
2.5962